In [4]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options

In [5]:
def Generate_Data(place = "Mumbai",Property_type = "Hostels",index=1):
    
    Dataset = pd.DataFrame(columns=['City',
                           'Property Type',
                           'Property Name',
                           'Contact Number',
                           'Ratings'])
    Dataset.to_csv(place+"_"+Property_type+".csv")
    
    place = place
    Property_type = Property_type
    index = index
    while True :
        print(index)
        if(index <=100):
            link = "https://www.justdial.com/"+place+"/"+Property_type+"/page-"+str(index) 
            print(link)
            next_disabled,Content = Scrap_Data(link,place,Property_type)
            #Content.to_csv(place+"_"+Property_type+"_"+str(index)+".csv")
            #Dataset.append(Content,ignore_index=True)
            Content.to_csv(place+"_"+Property_type+".csv",mode='a', header=False)
            if(next_disabled):
                break
            else:
                index += 1
        else:
            break
    return Dataset
    
    

In [6]:
def Scrap_Data(link,place,Property_type):
    
    #dataset = pd.DataFrame(columns=['City','Property Type','Contact Number','Rating'])
    
    City = []
    Property = []
    Contact_Number = []
    Rating = []
    Name = []
    
    Last_Page = False
    
    #print(Property_type)
    link = link
    
    dict_icon_number = {'icon-dc':'+',
                        'icon-fe':'(',
                        'icon-hg':')',
                        'icon-ba':'-',
                        'icon-acb':'0',
                        'icon-yz':'1',
                        'icon-wx':'2',
                        'icon-vu':'3',
                        'icon-ts':'4',
                        'icon-rq':'5',
                        'icon-po':'6',
                        'icon-nm':'7',
                        'icon-lk':'8',
                        'icon-ji':'9',}
    
    
    
    if(link):
        
        url = link
        #driver = webdriver.Firefox()
        chrome_options = Options()
        #chrome_options.add_argument("--disable-extensions")
        #chrome_options.add_argument("--disable-gpu")
        #chrome_options.add_argument("--no-sandbox") # linux only
        chrome_options.add_argument("--headless")
        # chrome_options.headless = True # also works
        driver = webdriver.Chrome(options=chrome_options)
            
        driver.get(url)
    
        SCROLL_PAUSE_TIME = 4

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        pg_div  = soup.find_all('div',class_='col-sm-5 col-xs-8 store-details sp-detail paddingR0')
        
        for container in pg_div:
            
            City.append(place)
            Property.append(Property_type)
            
            #Hostel Name
            hostel_name = container.h2.span.a['title']
            Name.append(hostel_name)
            #print("Hostel Name : "+hostel_name)
            
            #Ratings
            rating = container.find('span',class_="green-box")
            if(rating):
                Rating.append(rating.text)
            else:
                Rating.append("0")
            #print("Rating : "+rating.text)
            
            #Mobile Number
            sub_container_mob_num = container.find('p',class_='contact-info')
            if(sub_container_mob_num):
                sub_container_mob_num = sub_container_mob_num.span
            else:
                #print("Contact Number : None")
                #print("\n")
                #print("----------------------------")
                Contact_Number.append(None)
                continue
            
            if(sub_container_mob_num.a.find('b')):
                sub_container_mob_num = sub_container_mob_num.a.b
            else:
                sub_container_mob_num = sub_container_mob_num.a
            
            
            Digits = []
            mobile_number = ''
            if(sub_container_mob_num):
                span_container = sub_container_mob_num.find_all('span')
                for s in span_container:
                    Digits.append(dict_icon_number[s['class'][1]])
                #print("Contact Number : "+''.join(map(str, Digits)) )
                mobile_number = ''.join(map(str, Digits))
            else:
                #print("Contact Number : None")
                pass
            Contact_Number.append(mobile_number) 
            #print("\n")
            #print("----------------------------")
            
        next_disabled = soup.find_all('a',{'class':'dis','rel':True})
        if(next_disabled):
            for item in next_disabled:
                if(item['rel'][0] == 'next'):
                    Last_Page = True
                    break

        
        
         
    Content = pd.DataFrame({
                           'City': City,
                           'Property Type': Property,
                           'Property Name':Name,
                           'Contact Number': Contact_Number,
                           'Ratings': Rating
                           })
    #print(len(Property))
    driver.quit()
    return Last_Page,Content


In [7]:
City = 'Pune'
Property_type = 'Hostels'
Dataset = Generate_Data(City,Property_type)
Dataset

1
https://www.justdial.com/Pune/Hostels/page-1


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [66]:
! ls


Delhi_Hostel.csv	Gurgaon_Hostel.csv	  Noida_PG.csv
Delhi_Paying Guest.csv	Gurgaon_Paying Guest.csv  Scraping JustDial.ipynb
Delhi_PG.csv		Gurgaon_PG.csv
geckodriver.log		Noida_Hostel.csv


In [78]:
Data = pd.read_csv('Noida_Hostel.csv')
Data

,Unnamed: 0,City,Property Type,Property Name,Contact Number,Ratings
0,0,Noida,Hostel,"Cubic Stays Pvt Ltd in Noida Sector 70, Delhi",+(91)-9152568767,5.0
1,1,Noida,Hostel,"Shivshakti Pg And Hostel in Noida Sector 128, ...",+(91)-9152470699,3.5
2,2,Noida,Hostel,"Vansh Pg For Boys in Noida Sector 56, Delhi",+(91)-9152366207,5.0
3,3,Noida,Hostel,"Blue Sapphire Hospitality in Noida Sector 61, ...",+(91)-9152501083,4.5
4,4,Noida,Hostel,"Anchal Pg Services in Noida Sector 39, Delhi",+(91)-9152886101,4.7
...,...,...,...,...,...,...
1244,95,Noida,Hostel,"R K Boys Hostel in Noida Sector 22, Delhi",+(91)-8077964667,0.0
1245,96,Noida,Hostel,"R.k Hostel in Noida Sector 22, Delhi",+(91)-9650492122,0.0
1246,97,Noida,Hostel,"Rakhi Boy Hostel in Nr C-12, Bahijee Mkt, Sec-...",+(91)-9899303993,0.0
1247,98,Noida,Hostel,"RAGHAV boys hostel in Noida Sector 8, Delhi",+(91)-7503395251,0.0


In [79]:
Data = Data[Data['Contact Number'].notna()]
Data = Data.drop_duplicates('Contact Number')

In [80]:
Data.drop(columns=['Unnamed: 0'],inplace=True)
Data
Data.to_csv('Noida_H.csv',index=False)

In [81]:
Data

,City,Property Type,Property Name,Contact Number,Ratings
0,Noida,Hostel,"Cubic Stays Pvt Ltd in Noida Sector 70, Delhi",+(91)-9152568767,5.0
1,Noida,Hostel,"Shivshakti Pg And Hostel in Noida Sector 128, ...",+(91)-9152470699,3.5
2,Noida,Hostel,"Vansh Pg For Boys in Noida Sector 56, Delhi",+(91)-9152366207,5.0
3,Noida,Hostel,"Blue Sapphire Hospitality in Noida Sector 61, ...",+(91)-9152501083,4.5
4,Noida,Hostel,"Anchal Pg Services in Noida Sector 39, Delhi",+(91)-9152886101,4.7
...,...,...,...,...,...
1140,Noida,Hostel,Om Sai Ram Goswami P G & Hostel in Near Sai Ma...,+(91)-9911859843,0.0
1141,Noida,Hostel,Om SAI Ram Goswami P G & Hostel Sector Sixty i...,+(91)-9871252382,0.0
1142,Noida,Hostel,"Pardeep hostel in Noida Sector 135, Delhi",+(91)-9355113511,0.0
1144,Noida,Hostel,"R K Boys Hostel in Noida Sector 22, Delhi",+(91)-8077964667,0.0


In [82]:
filenames = ['Delhi_H.csv','Gurgaon_H.csv','Noida_H.csv']
li = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv('DelhiNCR_Hostel.csv',index=False)

In [83]:
frame

,City,Property Type,Property Name,Contact Number,Ratings
0,Delhi,Hostel,"Foresta Home in Laxmi Nagar, Delhi",+(91)-9152478001,3.8
1,Delhi,Hostel,"NRI Hostel Pg Services in Pari Chowk, Delhi",+(91)-9871806042,4.7
2,Delhi,Hostel,"Vidhya Niwas Paying Guest in Malka Ganj, Delhi",+(91)-9152410122,3.9
3,Delhi,Hostel,"Preet Pg For Girls in Vasant Kunj, Delhi",+(91)-9650360176,4.9
4,Delhi,Hostel,"Om Sai Ram Girls Pg in Greater Noida, Delhi",+(91)-9152453989,4.0
...,...,...,...,...,...
1291,Noida,Hostel,Om Sai Ram Goswami P G & Hostel in Near Sai Ma...,+(91)-9911859843,0.0
1292,Noida,Hostel,Om SAI Ram Goswami P G & Hostel Sector Sixty i...,+(91)-9871252382,0.0
1293,Noida,Hostel,"Pardeep hostel in Noida Sector 135, Delhi",+(91)-9355113511,0.0
1294,Noida,Hostel,"R K Boys Hostel in Noida Sector 22, Delhi",+(91)-8077964667,0.0


In [85]:
frame = pd.read_csv('/home/kapil/Desktop/DelhiNCR_PG.csv')
frame

,City,Property Type,Property Name,Contact Number,Ratings
0,Delhi,Paying Guest,"Prashant Homez in Noida Sector 66, Delhi",+(91)-9152455609,4.8
1,Delhi,Paying Guest,Wow The Sai Aashray Female Pg Accommodation in...,+(91)-9152501612,4.6
2,Delhi,Paying Guest,"Pushpanjali Pg in Dwarka Sector 7, Delhi",+(91)-9152851672,5.0
3,Delhi,Paying Guest,"Sara Home in Chattarpur, Delhi",+(91)-9152901049,3.5
4,Delhi,Paying Guest,"Vidya Group Of Hostels in Easy day, Delhi",+(91)-9152326234,3.5
...,...,...,...,...,...
1497,Noida,Paying Guest,"Paying Gest in A Block, Delhi",+(91)-9811163778,0.0
1498,Noida,Paying Guest,"Paying Developer Technology in Tughlakabad, Delhi",+(91)-8527221087,0.0
1499,Noida,Paying Guest,"PAYING AKHANCHA in , Delhi",+(91)-120-6444556,3.0
1500,Noida,Paying Guest,"paying uest Universe in Indirapuram, Delhi",+(91)-9821139245,0.0
